# 模板特化与SFINAE - 实践篇

本notebook通过实际代码帮助你理解C++模板特化和SFINAE机制。

**学习目标：**
- 掌握全特化与偏特化的使用
- 理解SFINAE原理
- 使用 `std::enable_if` 实现条件编译
- 使用 `std::void_t` 实现类型检测


## 1. 模板全特化

为特定类型提供完全不同的实现。


In [ ]:
%%writefile full_specialization.cpp
#include <iostream>
#include <cstring>

// 通用模板：类型信息
template <typename T>
struct TypeInfo {
    static const char* name() { return "unknown"; }
    static bool isNumeric() { return false; }
};

// 全特化：int类型
template <>
struct TypeInfo<int> {
    static const char* name() { return "int"; }
    static bool isNumeric() { return true; }
};

// 全特化：double类型
template <>
struct TypeInfo<double> {
    static const char* name() { return "double"; }
    static bool isNumeric() { return true; }
};

// 全特化：bool类型
template <>
struct TypeInfo<bool> {
    static const char* name() { return "bool"; }
    static bool isNumeric() { return false; }
};

// 函数模板全特化
template <typename T>
T maxValue(T a, T b) {
    return (a > b) ? a : b;
}

// 针对C字符串的全特化
template <>
const char* maxValue<const char*>(const char* a, const char* b) {
    return (std::strcmp(a, b) > 0) ? a : b;
}

int main() {
    std::cout << "==========================================" << std::endl;
    std::cout << "           模板全特化演示" << std::endl;
    std::cout << "==========================================\n" << std::endl;
    
    // 类模板特化
    std::cout << "【TypeInfo 全特化】" << std::endl;
    std::cout << "TypeInfo<int>::name() = " << TypeInfo<int>::name() << std::endl;
    std::cout << "TypeInfo<double>::name() = " << TypeInfo<double>::name() << std::endl;
    std::cout << "TypeInfo<bool>::name() = " << TypeInfo<bool>::name() << std::endl;
    std::cout << "TypeInfo<char>::name() = " << TypeInfo<char>::name() << " (未特化)" << std::endl;
    
    std::cout << "\nTypeInfo<int>::isNumeric() = " << TypeInfo<int>::isNumeric() << std::endl;
    std::cout << "TypeInfo<bool>::isNumeric() = " << TypeInfo<bool>::isNumeric() << std::endl;
    
    // 函数模板特化
    std::cout << "\n【maxValue 函数特化】" << std::endl;
    std::cout << "maxValue(10, 20) = " << maxValue(10, 20) << std::endl;
    std::cout << "maxValue(\"apple\", \"banana\") = " << maxValue("apple", "banana") << std::endl;
    std::cout << "maxValue(\"zoo\", \"apple\") = " << maxValue("zoo", "apple") << std::endl;
    
    return 0;
}


In [ ]:
!g++ -std=c++17 -o full_specialization full_specialization.cpp && ./full_specialization


## 2. 模板偏特化

针对一类类型（如所有指针类型）提供特殊实现。


In [ ]:
%%writefile partial_specialization.cpp
#include <iostream>

// 通用模板
template <typename T>
class Container {
    T value;
public:
    Container(T v) : value(v) {}
    void describe() {
        std::cout << "Container holding a value" << std::endl;
    }
    T get() { return value; }
};

// 偏特化：针对所有指针类型
template <typename T>
class Container<T*> {
    T* ptr;
public:
    Container(T* p) : ptr(p) {}
    void describe() {
        std::cout << "Container holding a POINTER" << std::endl;
    }
    T* get() { return ptr; }
    T deref() { return *ptr; }  // 特有方法：解引用
};

// 偏特化：针对数组类型
template <typename T, size_t N>
class Container<T[N]> {
    T arr[N];
public:
    Container() {}
    void describe() {
        std::cout << "Container holding an ARRAY of size " << N << std::endl;
    }
    size_t size() { return N; }
};

// ==================== 两参数模板的偏特化 ====================

template <typename T, typename U>
class Pair {
public:
    static void info() {
        std::cout << "Pair<T, U>: 两个不同类型" << std::endl;
    }
};

// 偏特化：两个类型相同
template <typename T>
class Pair<T, T> {
public:
    static void info() {
        std::cout << "Pair<T, T>: 两个相同类型" << std::endl;
    }
};

// 偏特化：第一个是指针
template <typename T, typename U>
class Pair<T*, U> {
public:
    static void info() {
        std::cout << "Pair<T*, U>: 第一个是指针" << std::endl;
    }
};

int main() {
    std::cout << "==========================================" << std::endl;
    std::cout << "           模板偏特化演示" << std::endl;
    std::cout << "==========================================\n" << std::endl;
    
    // Container偏特化
    std::cout << "【Container 偏特化】" << std::endl;
    
    int x = 42;
    Container<int> c1(x);
    c1.describe();
    
    Container<int*> c2(&x);
    c2.describe();
    std::cout << "  解引用值: " << c2.deref() << std::endl;
    
    Container<double[5]> c3;
    c3.describe();
    
    // Pair偏特化
    std::cout << "\n【Pair 偏特化】" << std::endl;
    Pair<int, double>::info();    // 通用模板
    Pair<int, int>::info();       // <T, T> 特化
    Pair<int*, double>::info();   // <T*, U> 特化
    
    return 0;
}


In [ ]:
!g++ -std=c++17 -o partial_specialization partial_specialization.cpp && ./partial_specialization


## 3. SFINAE 与 enable_if

SFINAE (Substitution Failure Is Not An Error) 允许在编译期根据类型特征选择不同的函数重载。


In [ ]:
%%writefile sfinae_demo.cpp
#include <iostream>
#include <type_traits>
#include <vector>
#include <string>

// ==================== 使用 enable_if 实现条件重载 ====================

// 只对整数类型启用
template <typename T>
std::enable_if_t<std::is_integral_v<T>, void>
process(T value) {
    std::cout << "Processing INTEGER: " << value << std::endl;
}

// 只对浮点类型启用
template <typename T>
std::enable_if_t<std::is_floating_point_v<T>, void>
process(T value) {
    std::cout << "Processing FLOAT: " << value << std::endl;
}

// 只对指针类型启用
template <typename T>
std::enable_if_t<std::is_pointer_v<T>, void>
process(T value) {
    std::cout << "Processing POINTER: " << *value << std::endl;
}

// ==================== 使用默认模板参数实现 enable_if ====================

template <typename T, 
          std::enable_if_t<std::is_arithmetic_v<T>, int> = 0>
T doubleIt(T value) {
    return value * 2;
}

// 对于非算术类型，可以提供不同实现或让其编译失败

// ==================== 类模板中使用 enable_if ====================

template <typename T, typename Enable = void>
class Calculator {
public:
    static void info() {
        std::cout << "Calculator for generic type" << std::endl;
    }
};

// 特化：只对数值类型
template <typename T>
class Calculator<T, std::enable_if_t<std::is_arithmetic_v<T>>> {
public:
    static void info() {
        std::cout << "Calculator for ARITHMETIC type" << std::endl;
    }
    static T add(T a, T b) { return a + b; }
    static T multiply(T a, T b) { return a * b; }
};

int main() {
    std::cout << "==========================================" << std::endl;
    std::cout << "        SFINAE 与 enable_if 演示" << std::endl;
    std::cout << "==========================================\n" << std::endl;
    
    // 条件重载
    std::cout << "【条件重载】" << std::endl;
    process(42);           // 整数版本
    process(3.14);         // 浮点版本
    int x = 100;
    process(&x);           // 指针版本
    
    // enable_if 在默认参数中
    std::cout << "\n【doubleIt 函数】" << std::endl;
    std::cout << "doubleIt(21) = " << doubleIt(21) << std::endl;
    std::cout << "doubleIt(3.5) = " << doubleIt(3.5) << std::endl;
    // doubleIt("hello"); // 编译错误：string不是arithmetic
    
    // 类模板特化
    std::cout << "\n【Calculator 类模板】" << std::endl;
    Calculator<int>::info();
    std::cout << "Calculator<int>::add(10, 20) = " << Calculator<int>::add(10, 20) << std::endl;
    Calculator<std::string>::info();  // 使用通用模板
    
    return 0;
}


In [ ]:
!g++ -std=c++17 -o sfinae_demo sfinae_demo.cpp && ./sfinae_demo


## 4. void_t 与类型检测

使用 `std::void_t` 检测类型是否具有特定成员或特征。


In [ ]:
%%writefile void_t_demo.cpp
#include <iostream>
#include <type_traits>
#include <vector>
#include <string>

// ==================== 检测 value_type 成员 ====================

template <typename T, typename = void>
struct has_value_type : std::false_type {};

template <typename T>
struct has_value_type<T, std::void_t<typename T::value_type>> : std::true_type {};

// ==================== 检测 size() 成员函数 ====================

template <typename T, typename = void>
struct has_size : std::false_type {};

template <typename T>
struct has_size<T, std::void_t<decltype(std::declval<T>().size())>> : std::true_type {};

// ==================== 检测 begin()/end() 成员函数 ====================

template <typename T, typename = void>
struct is_iterable : std::false_type {};

template <typename T>
struct is_iterable<T, std::void_t<
    decltype(std::declval<T>().begin()),
    decltype(std::declval<T>().end())
>> : std::true_type {};

// ==================== 根据类型特征选择实现 ====================

template <typename T>
void printInfo(const T& obj) {
    std::cout << "Type analysis for " << typeid(T).name() << ":" << std::endl;
    std::cout << "  has_value_type: " << has_value_type<T>::value << std::endl;
    std::cout << "  has_size: " << has_size<T>::value << std::endl;
    std::cout << "  is_iterable: " << is_iterable<T>::value << std::endl;
}

// 只对可迭代类型启用的函数
template <typename T>
std::enable_if_t<is_iterable<T>::value>
printElements(const T& container) {
    std::cout << "Elements: ";
    for (const auto& elem : container) {
        std::cout << elem << " ";
    }
    std::cout << std::endl;
}

int main() {
    std::cout << "==========================================" << std::endl;
    std::cout << "        void_t 类型检测演示" << std::endl;
    std::cout << "==========================================\n" << std::endl;
    
    std::cout << "【类型特征检测】" << std::endl;
    
    std::cout << "\nstd::vector<int>:" << std::endl;
    printInfo(std::vector<int>{});
    
    std::cout << "\nstd::string:" << std::endl;
    printInfo(std::string{});
    
    std::cout << "\nint:" << std::endl;
    printInfo(42);
    
    // 使用检测结果
    std::cout << "\n【根据类型特征选择行为】" << std::endl;
    std::vector<int> vec = {1, 2, 3, 4, 5};
    printElements(vec);
    
    std::string str = "Hello";
    printElements(str);
    
    // printElements(42);  // 编译错误：int不可迭代
    
    return 0;
}


In [ ]:
!g++ -std=c++17 -o void_t_demo void_t_demo.cpp && ./void_t_demo


## 清理临时文件


In [ ]:
!rm -f full_specialization.cpp full_specialization partial_specialization.cpp partial_specialization sfinae_demo.cpp sfinae_demo void_t_demo.cpp void_t_demo


## 总结

通过本notebook，你应该理解了：

1. **全特化**
   - 为特定类型提供完全不同的实现
   - 语法：`template <> class C<int> { ... }`

2. **偏特化**
   - 为一类类型（如所有指针）提供特殊实现
   - 函数模板不支持偏特化，使用重载代替

3. **SFINAE**
   - 替换失败不是错误，编译器会尝试其他重载
   - 使用 `enable_if` 控制函数重载的选择

4. **void_t**
   - 用于检测类型是否具有特定成员
   - 结合 `enable_if` 实现类型安全的泛型编程

## 练习

1. 实现一个 `is_container` 类型特征，检测类型是否有 `begin()`, `end()`, `size()`
2. 使用 SFINAE 实现一个 `toString` 函数，对不同类型采用不同的转换策略
3. 阅读 FlashAttention 中的 `static_switch.h`，理解其宏实现原理
